In [1]:
import pandas as pd
import numpy as np
import sklearn
import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn import neighbors

TRAIN_DATA_PATH = "../datasets/agoda_cancellation_train.csv"
REAL_TEST_PATH = "../datasets/test_set_week_1.csv"
OUTPUT_PATH = "../output/week1_pred.csv"
original_data = pd.read_csv(TRAIN_DATA_PATH, index_col=0).drop_duplicates()

In [2]:
def cancellaction_cost(days_before, cancellation_policy):
#     days_before = (checkin_date - base_date).days
    cost = 100
    for policy in cancellation_policy.split("_"):
        if "D" in policy:
            if int(policy.split("D")[0]) <= days_before:
                if policy.split("D")[1][-1] == "N":
                    # treat 1 night as 40 percent - change this
                    return 40
                elif policy.split("D")[1][-1] == "P":
                    return int(policy.split("D")[1][:-1])
    return cost

def filtered_data_by_date(data, base_date_t):
    days_after_date_to_cancel = 7 
    min_days_after_booking = 6 # 8
    max_days_after_booking = 35 # 40
    min_days_to_checkin = 8 # 9
    processed_data = data.copy()

    processed_data["days_before_checkin"] = (pd.to_datetime(processed_data['checkin_date']) - base_date_t).dt.days
    processed_data["days_after_booking"] = (base_date_t - pd.to_datetime(processed_data['booking_datetime'])).dt.days
        
    processed_data['cancel_cost'] = processed_data.apply(lambda x: 
                                                         cancellaction_cost(int(x["days_before_checkin"]), x['cancellation_policy_code']), axis=1)
    processed_data['cancel_cost_changes'] = processed_data.apply(lambda x: 
                                                         0 if x["cancel_cost"] == cancellaction_cost(max(0, int(x["days_before_checkin"]) - 7), x['cancellation_policy_code']) else 1, axis=1)
        
    
    processed_data = processed_data[processed_data["days_before_checkin"] > min_days_to_checkin]
    processed_data = processed_data[processed_data["days_after_booking"].between(min_days_after_booking, max_days_after_booking, inclusive="both")]
    
    y = None
    if 'cancellation_datetime' in processed_data:
        processed_data["days_to_cancel"] = (pd.to_datetime(processed_data['cancellation_datetime']) - base_date_t).dt.days
        y = processed_data["days_to_cancel"].between(0, days_after_date_to_cancel)
    
    return processed_data, y

# train_x, train_y = filtered_data_by_date(original_data, datetime.datetime(2018, 6, 21, 0, 0))
# test_x, test_y = filtered_data_by_date(original_data, datetime.datetime(2018, 8, 30, 0, 0))


t = datetime.datetime(2018, 6, 7, 0, 0) # weekday is the same as (2018, 12, 7, 0, 0)
train_x, train_y = filtered_data_by_date(original_data, datetime.datetime(2018, 6, 7, 0, 0))
while t < datetime.datetime(2018, 7, 27, 0, 0):
    tmp_x, tmp_y = filtered_data_by_date(original_data, t)
    train_x = pd.concat([train_x, tmp_x])
    train_y = pd.concat([train_y, tmp_y])
    t = t + datetime.timedelta(days=7)
test_x, test_y = filtered_data_by_date(original_data, datetime.datetime(2018, 8, 30, 0, 0))

# train without duplicates
# train_x = train_x.groupby('h_booking_id').first()
# train_y = train_y.groupby('h_booking_id').first()

print(len(train_y), sum(train_y), len(test_y), sum(test_y))
print(len(train_x), len(test_x), len(test_x.index.intersection(train_x.index)))
train_x[train_y]

36138 1356 2276 123
36138 2276 0


,booking_datetime,checkin_date,checkout_date,hotel_id,hotel_country_code,hotel_live_date,hotel_star_rating,accommadation_type_name,charge_option,h_customer_id,...,cancellation_datetime,hotel_area_code,hotel_brand_code,hotel_chain_code,hotel_city_code,days_before_checkin,days_after_booking,cancel_cost,cancel_cost_changes,days_to_cancel
h_booking_id,,,,,,,,,,,,,,,,,,,,,
-9158802160721656486,2018-05-14 21:05:00,2018-07-27 00:00:00,2018-07-29 00:00:00,1619979,TH,2016-11-09 11:49:00,3.0,Hotel,Pay Later,4863240163489780019,...,2018-06-13,3303,NaN,NaN,209,50,23,100,0,6.0
-9051647732738994334,2018-05-21 23:53:00,2018-08-21 00:00:00,2018-08-25 00:00:00,3091138,TH,2017-11-02 13:22:00,4.0,Resort,Pay Now,3389767402530290086,...,2018-06-12,27,NaN,NaN,2180,75,16,100,0,5.0
-8670473951737824795,2018-05-12 11:40:00,2018-08-10 00:00:00,2018-08-11 00:00:00,774305,JP,2014-11-17 14:22:00,4.0,Hotel,Pay Now,7667673889030460094,...,2018-06-10,1741,NaN,NaN,1403,64,25,50,0,3.0
-8575692963466322400,2018-05-15 23:18:00,2018-07-03 00:00:00,2018-07-08 00:00:00,315738,GB,2012-05-15 15:55:00,0.0,Hostel,Pay Later,5529213351593640073,...,2018-06-14,930,NaN,NaN,2538,26,22,40,0,7.0
-8538979200143786968,2018-05-11 16:17:00,2018-07-21 00:00:00,2018-07-22 00:00:00,3096224,TW,2017-11-02 14:16:00,5.0,Resort,Pay Now,6339307434454840085,...,2018-06-14,4161,NaN,NaN,96,44,26,100,0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8628846012636740560,2018-07-11 20:29:00,2018-08-07 00:00:00,2018-08-11 00:00:00,2599741,JP,2017-08-18 16:18:00,3.0,Hotel,Pay Later,1071836652144100056,...,2018-07-31,2298,NaN,NaN,1128,12,14,100,0,5.0
8870967893939751388,2018-07-17 21:50:00,2018-09-07 00:00:00,2018-09-09 00:00:00,1772045,TW,2017-01-10 19:30:00,3.0,Hotel,Pay Later,1002820500766480087,...,2018-07-31,2969,NaN,NaN,162,43,8,40,0,5.0
8879128375080305810,2018-07-14 14:40:00,2018-09-06 00:00:00,2018-09-11 00:00:00,281014,JP,2011-07-23 10:04:00,3.0,Resort,Pay Later,475276544743910090,...,2018-08-01,1366,NaN,NaN,2187,42,11,50,0,6.0


In [3]:
def get_features(basic_data):
    processed_data = basic_data.copy()
    processed_data["booking_length"] = (pd.to_datetime(processed_data['checkout_date']) - pd.to_datetime(processed_data['checkin_date'])).dt.days
    processed_data["booking_checkin_diff"] = (pd.to_datetime(processed_data['checkin_date']) - pd.to_datetime(processed_data['booking_datetime'])).dt.days
    processed_data["hotel_time_in_system"] = (pd.to_datetime(processed_data['hotel_live_date']) - pd.to_datetime(processed_data['booking_datetime'])).dt.days
    processed_data["is_user_logged_in"] = processed_data['is_user_logged_in'].apply(lambda x: 1 if x else 0)
    processed_data["is_first_booking"] = processed_data['is_first_booking'].apply(lambda x: 1 if x else 0)
    
    charge_option_dummies = pd.get_dummies(processed_data["charge_option"], prefix="charge")
    cancel_policy_dummies = pd.get_dummies(processed_data["cancellation_policy_code"], prefix="cancelp")
    acc_type_dummies = pd.get_dummies(processed_data["accommadation_type_name"], prefix="acc_type")
    city_code_dummies = pd.get_dummies(processed_data["hotel_city_code"], prefix="city_code")
    payment_type_dummies = pd.get_dummies(processed_data["original_payment_type"], prefix="payment_type")
    payment_method_dummies = pd.get_dummies(processed_data["original_payment_method"], prefix="payment_method")
    
    final_data = processed_data[["days_before_checkin", "days_after_booking", "booking_length", "booking_checkin_diff",
                                 "is_first_booking", "is_user_logged_in", "cancel_cost",
                                "hotel_star_rating", "cancel_cost_changes"
#                                 "hotel_time_in_system", 
                                ]]
    
    final_data = pd.concat([final_data, charge_option_dummies], axis=1)
    final_data = pd.concat([final_data, cancel_policy_dummies], axis=1)
    final_data = pd.concat([final_data, acc_type_dummies], axis=1)
#     final_data = pd.concat([final_data, city_code_dummies], axis=1)
#     final_data = pd.concat([final_data, payment_type_dummies], axis=1)
    final_data = pd.concat([final_data, payment_method_dummies], axis=1)
    
    final_data["guest_is_not_the_customer"] = processed_data['guest_is_not_the_customer'].apply(lambda x: 1 if x else 0)
    
#     final_data["no_of_adults"] = processed_data['no_of_adults'].apply(lambda x: x if x else 0)
#     final_data["no_of_children"] = processed_data['no_of_children'].apply(lambda x: x if x else 0)
#     final_data["no_of_extra_bed"] = processed_data['no_of_extra_bed'].apply(lambda x: x if x else 0)
#     final_data["no_of_room"] = processed_data['no_of_room'].apply(lambda x: x if x else 0)
#     final_data["original_selling_amount"] = processed_data['original_selling_amount'].apply(lambda x: x if x else 0)
#     final_data["request_nonesmoke"] = processed_data['request_nonesmoke'].apply(lambda x: 1 if x else 0)
#     final_data["request_latecheckin"] = processed_data['request_latecheckin'].apply(lambda x: 1 if x else 0)
#     final_data["request_highfloor"] = processed_data['request_highfloor'].apply(lambda x: 1 if x else 0)
#     final_data["request_largebed"] = processed_data['request_largebed'].apply(lambda x: 1 if x else 0)
#     final_data["request_twinbeds"] = processed_data['request_twinbeds'].apply(lambda x: 1 if x else 0)
#     final_data["request_airport"] = processed_data['request_airport'].apply(lambda x: 1 if x else 0)
#     final_data["request_earlycheckin"] = processed_data['request_earlycheckin'].apply(lambda x: 1 if x else 0)

    return final_data

train_x_features = get_features(train_x)
test_x_features = get_features(test_x)
test_x_features = test_x_features.reindex(columns = train_x_features.columns, fill_value=0)

clf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)# , max_features=10)


clf.fit(train_x_features, train_y)

prediction = clf.predict(test_x_features)

train_test_pred_prob = clf.predict_proba(test_x_features) 
prediction = train_test_pred_prob[:,1] >= np.percentile(train_test_pred_prob[:,1], 96)

true_positive = sum([1 for i in range(len(prediction)) if 1 == prediction[i] == test_y.iloc[i]])
true_negative = sum([1 for i in range(len(prediction)) if 0 == prediction[i] == test_y.iloc[i]])
real_positive = sum([1 for i in test_y if i == 1])
real_negative = sum([1 for i in test_y if i == 0])

print(f"trained on {len(train_x)}")

print(f"True Positive: {true_positive}/{real_positive}")
print(f"True Negative: {true_negative}/{real_negative}")
print(f"Total True: {true_positive+true_negative}/{real_positive+real_negative} = {(true_positive+true_negative)/(real_positive+real_negative)}")

train_test_pred_prob = clf.predict_proba(test_x_features) 
print(f"AUC Score: {roc_auc_score(test_y, train_test_pred_prob[:,1])}")

force_true = train_test_pred_prob[:,1] >= np.percentile(train_test_pred_prob[:,1], 96)
print(f"If forced true: change {np.percentile(train_test_pred_prob[:,1], 96)} {sum(force_true)}, {sum(test_y[force_true])} real positive")
print(min(train_test_pred_prob[:,1]), max(train_test_pred_prob[:,1]))

# print(f"Mock score {roc_auc_score(test_y, np.array([i/len(test_y) for i in range(len(test_y))]))}")
train_test_pred_prob

trained on 36138
True Positive: 12/123
True Negative: 2073/2153
Total True: 2085/2276 = 0.9160808435852372
AUC Score: 0.6954089396908832
If forced true: change 0.057368029383164904 92, 12 real positive
0.019944687004966717 0.11439690272011833


array([[0.96859496, 0.03140504],
       [0.96320904, 0.03679096],
       [0.95482169, 0.04517831],
       ...,
       [0.96386471, 0.03613529],
       [0.96107128, 0.03892872],
       [0.94361719, 0.05638281]])

In [4]:
real_test_data = pd.read_csv(REAL_TEST_PATH, index_col=0).drop_duplicates()

real_test_data_x, _ = filtered_data_by_date(real_test_data, datetime.datetime(2018, 12, 7, 0, 0))
real_test_data_features = get_features(real_test_data_x)

assert len(real_test_data_features) == len(real_test_data)
real_test_data.loc[real_test_data.index.difference(real_test_data_features.index)]


real_test_data_features = real_test_data_features.reindex(columns=train_x_features.columns, fill_value=0)
print(sum(clf.predict(real_test_data_features)))

predict_prob = clf.predict_proba(real_test_data_features)
print(min(predict_prob[:,1]), max(predict_prob[:,1]))

forced_predictions = predict_prob[:,1] >= np.percentile(predict_prob[:,1], 96)
pd.DataFrame(list(map(int, forced_predictions)), columns=["predicted_values"]).to_csv(OUTPUT_PATH, index=False)

real_test_data_features[forced_predictions]

0
0.019137416613055607 0.07969347127588405


,days_before_checkin,days_after_booking,booking_length,booking_checkin_diff,is_first_booking,is_user_logged_in,cancel_cost,hotel_star_rating,cancel_cost_changes,charge_Pay Later,...,payment_method_PayPal,payment_method_RHB Now,payment_method_SCB,payment_method_Tesco Lotus,payment_method_UNKNOWN,payment_method_UnionPay - Creditcard,payment_method_UnionPay - Debit Card,payment_method_Visa,payment_method_WeChat,guest_is_not_the_customer
h_booking_id,,,,,,,,,,,,,,,,,,,,,
-9215699014988905507,9,16,4,25,1,0,100,5.0,0,0,...,0,0,0,0,0,0,0,0,0,0
-9212818888388326328,17,8,2,25,0,0,50,5.0,0,1,...,0,0,0,0,0,0,0,1,0,0
-9156852661231038901,10,17,1,27,0,1,30,3.0,0,1,...,0,0,0,0,0,0,0,0,0,0
-9131161804281079496,10,6,4,16,1,0,14,4.0,1,1,...,0,0,0,0,0,0,0,0,0,0
-9033940594379411553,17,7,2,24,0,1,40,3.0,0,0,...,0,0,0,0,0,0,0,0,0,0
-8993817633097420526,11,12,1,23,0,1,40,3.0,0,1,...,0,0,0,0,0,0,0,1,0,0
-8987127384223575963,9,11,1,20,0,1,40,2.0,1,1,...,0,0,0,0,0,0,0,1,0,1
-8920800619480008881,22,9,1,31,0,1,40,3.0,0,1,...,0,0,0,0,0,0,0,0,0,0
-8908452601294778728,43,6,1,49,0,1,50,3.0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [5]:
#### This is only to check the pearson value of each column

pearsons = []

x = train_x
y = train_y

features = get_features(x)
for count, col_name in enumerate(features):
    col_x = features[col_name]
    
    # calculate pearson correlation
    # based on https://stackabuse.com/calculating-pearson-correlation-coefficient-in-python-with-numpy/
    sigma_x = np.sqrt(np.mean(col_x**2) - np.mean(col_x)**2)
    sigma_y = np.sqrt(np.mean(y**2) - np.mean(y)**2)
    sigma_xy = np.mean(col_x * y) - np.mean(col_x) * np.mean(y)
    pearson = sigma_xy / (sigma_x * sigma_y)
#     print(col_name, pearson)
    pearsons.append((col_name, pearson))

print(sorted(pearsons, key=lambda x: abs(x[1]), reverse=True))

[('cancelp_365D100P_100P', -0.10014497613042886), ('charge_Pay Later', 0.08681264079584097), ('charge_Pay Now', -0.08681264079584095), ('booking_checkin_diff', -0.05948756552510412), ('cancel_cost', -0.050008964235787795), ('days_after_booking', -0.047749760210531594), ('days_before_checkin', -0.047031718794630945), ('cancelp_10D100P', 0.0348172836450045), ('is_user_logged_in', 0.03251377679711981), ('cancelp_1D56P_0D100P_100P', 0.030165014490296467), ('cancelp_8D51P_100P', 0.030165014490296467), ('cancelp_1D1N_1N', 0.02984843393876077), ('booking_length', 0.02876921396183836), ('is_first_booking', -0.0282655876716125), ('hotel_star_rating', -0.027330074430540776), ('cancelp_18D100P', 0.02664227966237114), ('cancelp_25D100P_100P', 0.02664227966237114), ('cancelp_3D17P_100P', 0.02664227966237114), ('cancelp_7D10P_3D50P_100P', 0.02560467505196415), ('cancelp_1D100P_100P', 0.025394237858573455), ('cancelp_2D100P', 0.02443488047491037), ('cancelp_5D100P_100P', 0.023092378753669568), ('canc